# Identifying Exoplanets using the TESS Objects of Interest Data Set
***Author: Matt Paterson***<br>
***Machine Learning Engineer***<br>
***Santa Cruz, California***<br>

<img src='https://hips.hearstapps.com/pop.h-cdn.co/assets/16/30/1280x907/gallery-1469724312-thetoolsneed.png?resize=400:*' style="float: left; margin: 20px; height: 250px"><img src='https://www.dailydot.com/wp-content/uploads/2018/04/exoplanet-missions.jpg' style="float: right; margin: 20px; height: 250px">

**Note about the Data**<br>
This data set is downloaded from the Cal Tech NASA Exoplanet Archive found <a href='https://exoplanetarchive.ipac.caltech.edu/cgi-bin/TblView/nph-tblView?app=ExoTbls&config=TOI'>here</a> and was downloaded on 2021-03-06 for the purpose of creating this instructional notebook.

**Note about the Spacecraft**<br>
TESS was launched by NASA on April 18, 2018 aboard a SpaceX Falcon-9 rocket, and it completed its initial mission in July of 2020, positively identifying 66 planets and recording over 2100 objects of interest. You can learn more about TESS <a href='https://exoplanets.nasa.gov/tess/'>here</a>

**Note about the Environment**<br>
If you normally like to code with music I highly recommend typing "Space Jazz" into the search field in your music streaming service of choice while you experience this Lab.

## The Data Science Problem
Can we use data from over 2000 observations made by the Transiting Exoplanet Survey Satellite (TESS) to positively identify more exoplanets in the sky using Machine Learning models?

## Import the necessary python libraries
In this notebook we will use python exclusively and in order to build our models, we'll utilize scikit-learn's libraries. This particular notebook will focus on the Logistic Regression model only.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model    import LogisticRegression
from sklearn.metrics         import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing   import StandardScaler
from sklearn.metrics         import plot_confusion_matrix

## Import your data
The data that we'll be using is already downloaded from CalTech/NASA, cleaned up a little for use here, and saved in a folder called 'data'. That 'data' folder lives in the same directory as a folder called 'code' where this notebook lives.

In order to import the data set, you'll have to provide the pandas function with a full datapath to find your csv file. You can choose to do this using the absolute path, which is recommended for production code, but for ease in this situation, you can go ahead and use the relative path.

In [2]:
datapath = '../data/'
filename = ???

Import the data set into a pandas DataFrame and save it in a variable called 'df', and go ahead and set the index column to 'rowid' using an argument in the pandas function. Then display 5 rows of data in your DataFrame.

In [1]:
# your_code_here:

How many rows and columns of data do you have in this data set? How many null values do you have in each column? Are the data types in the table logical and ready for modeling? Why or why not? What are some different python or pandas or matplotlib functions that you can use to inspect this data?

In [2]:
# your_code_here:

In [3]:
# your_code_here:

In [4]:
# your_code_here:

In [5]:
# your_code_here:


## Quick and dirty data cleaning
With any dataset, you'll want to spend lots of time looking at each and every column of data and learning what each is and how it relates to the other columns. You'll employ correlation matrices, scatterplots, and bar charts to see patterns. You'll have to impute missing data and decide if some data can be imputed at all. You'll have to engineer data columns and aggregate data columns and convert columns to different units of measurement. 

<img src='https://static.dvidshub.net/media/thumbs/photos/1007/297427/1000w_q95.jpg' style="float: left; margin: 20px; height: 250px">

If you want to really use this particular data set effectively, I highly recommend spending at least 7 minutes reading and exploring <a href='https://exoplanets.nasa.gov/alien-worlds/ways-to-find-a-planet/'>this website</a> for more background on identifying exoplanets.

### You're probably here to learn how to build a Logistic Regression Model...so to save time-->
This particular notebook is intended to help you learn how to use the Logistic Regression Classification Model, so rather than spend all of our time on the data (a typical Data Scientist spends most of their time acquiring, cleaning, and maintaining their data), we're going to make what I like to call the 'Quick and Dirty Model' by dropping all columns with null values.

## Drop all null values
Use a pandas method to drop all of the columns in your table that contain null values, and save this new dataframe to a variable called 'tess', and then print the number of rows and columns that the new 'tess' DataFrame contains.

In [7]:
tess = df.dropna(axis=1)
tess.shape

(2542, 37)

## Add back our target column
One of the columns that we dropped is actually VERY important to our model. df['tfopwg_disp'] is actually going to be our TARGET column, or the column that we're going to use for classification.

Use pd.merge() to add that column back on to our table, minding the index so that we don't switch anything around and completely break our model before we build it.

In [8]:
tess = pd.merge(tess, df['tfopwg_disp'], how='outer', left_index=True, right_index=True)

In [9]:
tess.shape

(2542, 38)

How many of your new columns are non-numeric?

In [ ]:
tess.dtypes

In [11]:
# display the type of object that this function returns
type(tess.dtypes)

pandas.core.series.Series

Calling the DataFrame's dtypes attribute returns a pandas Series, list-like object, the values of which are all strings displaying the datatype of each column in your DataFrame. Can you figure out a way to display the names of just the columns that are non-numeric?

In [12]:
# HINT: You can see the index of each row of a pandas Series in the
# same way that you list the index of each row in a pandas DataFrame
tess.dtypes.index

Index(['toi', 'toipfx', 'tid', 'ctoi_alias', 'pl_pnum', 'rastr', 'ra',
       'decstr', 'dec', 'pl_tranmid', 'pl_tranmidlim', 'pl_tranmidsymerr',
       'pl_orbperlim', 'pl_orbpersymerr', 'pl_trandurh', 'pl_trandurhlim',
       'pl_trandurhsymerr', 'pl_trandep', 'pl_trandeplim', 'pl_trandepsymerr',
       'pl_radelim', 'pl_radesymerr', 'st_tmag', 'st_tmagerr1', 'st_tmagerr2',
       'st_tmaglim', 'st_tmagsymerr', 'st_distlim', 'st_distsymerr',
       'st_tefflim', 'st_teffsymerr', 'st_logglim', 'st_loggsymerr',
       'st_radlim', 'st_radsymerr', 'toi_created', 'rowupdate', 'tfopwg_disp'],
      dtype='object')

If you want a short-cut, you can always just manually copy/paste the non-numeric column names, but that's not really a shortcut option when you have more than four non-numeric columns.

Note: Don't spend all of your time writing this function--the purpose of this notebook is to learn Logistic Regression using Python

In [8]:
#create an empty dictionary for your dtypes

#iterate through the Series of tess.dtypes and 
#create your dictionary using the tess.dtypes.index Series as keys

# print the non-numeric column names


## Data Engineering
What are these non-numeric columns? Are they important to our model? If so, how can we make them into numeric columns? 

There is a data dictionary in the data folder where we got the data set, and it should be helpful here. For more detailed information you can follow the link to the data set above and click on the "view documentation" button to see what these are.

In [ ]:
# type your answer here:

Display the data from a 7 rows of each of the non-numeric columns

In [7]:
# your_code_here:

What did you discover? Are these fields things that can be dropped?

## Drop these columns
We want to keep the 'tfopwg_disp' column as this will be our target column for our model, but we can drop the other four columns. 

We can drop them using the pd.DataFrame.drop() method, and we can make that permanent by dropping them "inplace" as we do.

In [ ]:
tess.shape

In [16]:
# your_code_here

In [ ]:
tess.shape

## What are we even predicting here?
We want to know if the dataset itself can help point out likely planetary candidates. To do this, we want to see what objects have already been identified by the CalTech and NASA scientists, and then infer commonalities in the data using Logistic Regression to classify the unclassified data points.

This means we'll want to convert the classifications that we have into a binary set of positive and negatively identified planets. Given the nature of our data set, there will be a third category of "unclassified" data, which is what we'll deploy our model on once it is trainied. Sounds easy, right?

### This key explains the basic classifications:
 	TESS Follow-up Observing Program Working Group (TFOPWG) Dispostion:

- CP=confirmed planet
- KP=known planet
- FP=false positive
- PC=planetary candidate 


<img src='https://wonderopolis.org/wp-content/uploads/2012/02/boy-scratching-head_shutterstock_68533339.jpg' style="float: center; margin: auto; height: 250px">


# warning - this might confuse you

The term "False Positive" will be used by us, Data Scientists and Machine Learning Engineers, to identify an observation that our model predicts should be a "Positive" but that the ground truth tells is was a "Negative". Since we **predicted** this to be "Positive" and we were wrong, this is known as a "False Positive".

The TESS Objects of Interest table was/is built by scientists who have a good reason to think that these objects are good candidates to be exoplanets. As such, if it turns out that through further investigation that one of these observations is not in fact an exoplanet, they classify that as a "false positive". In other words, it passed the pre-tests but failed the final evaluation. To keep things more clear for us in this context, we're going to convert that classification to a simple negative designation.

For ease of understanding, we'll create a mapping function to convert our designations to either 1 for exoplanet, 0 for not-exoplanet, or 2 for 'unclassified'.

In [ ]:
# use the pd.Series.value_counts() function to look at our baseline split
tess.tfopwg_disp.value_counts(normalize=True)

In [19]:
# create a dict that maps the classifications in TOI 
# to the classifications that we'll use for modeling

class_map = {
    'PC':2,
    'FP':0,
    'KP':1,
    'APC':2,
    'CP':1,
    'FA':0
}

In [20]:
tess['target'] = tess['tfopwg_disp'].map(class_map)

We also know that there are null fields in the target column. Since we know that no other fields are null in our dataframe, we can use the pd.DataFrame.fillna() funciton to change those NaN values to 'unclassified', or 2.

In [21]:
tess = tess.fillna(2)

Now we can look at our baseline model numbers

In [ ]:
tess.target.value_counts(normalize=True)

From our original data set, 63.8% of the data are unclassified, 19.7% of the data are negatively classified, and 16.5% are positively classified.

To train the model, we'll split off the unclassified data so that we're only training on classified data.

In [ ]:
tess_model = tess[tess['target'] != 2]
tess_model.target.value_counts(normalize=True)

So this baseline score of 54.3% is what our model has to beat. If our we guessed that every object we saw was a planet, we'd be wrong 54.3% of the time (and a converse hypothesis would result in that maximum rate).

## Train Test Split
We'll want to do more feature engineering to our dataset, but before we do, let's put down a couple of baselines by which to measure our progress. To do that, we're going to first see the distribution of the classfications in our target column, and then we'll use train_test_split to break up our data set into a training set and a testing set

In [ ]:
tess_model = tess_model.drop(columns='tfopwg_disp')
tess_model.shape

In [25]:
# Create your X matrix and y vector
X = tess_model.drop(columns='target')
y = tess_model['target']

# Use train_test_split() to split up your data set
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=.2, 
                                                    random_state=42)

<img src= 'https://miro.medium.com/max/640/0*gKOV65tvGfY8SMem.png' style="float: center; margin: auto; height: 250px">

## Model Time!!

Linear Regression Models are handy for predicting a continuous variable givn a set of data that is generally linear, normally distributed, and based on heteroscedastic variables. The data science problem that we are trying to solve today asks us to classify our data into one of two possible classifications: An observation is either an Exoplanet, or it is Not an Exoplanet. Thus, the normal line of best-fit will not be helpful.

Instead, we're going to try and fit our data to a Sigmoid curve using the Logistic Regression model. In short, instead of predicting a continuous value for our target column, this model will predict a binary value of 0 or 1 (it can be used for multi-class problems, but for now just focus on binary classification).

## How do we get to the prediction?

The model will assign a numerical probability for each observation between 0 and 1 (inclusive). In scikit-learn we use the function <code>model.predict_proba()</code> to view it. 

Using the Logit Function, the model is assigning a number based on the probability that the item is in one or the other of our two classes. If the probability is less than 0.5, then the observation is considered 0 (negative). Otherwise, it is considered 1 (positive).

## Ready to start the Experiment?<br><br>
<img src = 'https://www.womanthology.co.uk/wp-content/uploads/2014/07/Female-scientist-800-x-400.jpg' style="float: center; margin: auto; height: 250px">
<br><br>
Instantiate a Logistic Regression Model and print out your training, testing, and overall accuracy scores.


In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
print('Training Accuracy Score : ', model.score(X_train, y_train))
print('Testing Accuracy Score  : ', model.score(X_test, y_test))
print('Overall Accuracy Score  : ', model.score(X, y))

## How did we do? 
Recall that our baseline model, or the basic split between our binary classification classes, was 54%. Was our Quick and Dirty model an improvement over our baseline? Why or why not?

[Hide this] No it was not. Our model performed about the same as the baseline model.

Is our Model well-fit? Is it showing signs of bias? Is it overfit? Why or why not?

<img src = 'https://i.pinimg.com/originals/fd/9f/c5/fd9fc5938f92dc1a8d8ad9db6667ea7b.jpg'>

## Scale our data
Just like you can't run an HO Scale Model Train set on a G Scale Model Train track, we really shouldn't be trying to make predictive numerical models using numbers on drastically different scales. Things like temperature conversions (think Farenheit versus Kelvin) or distance scales (think millimeters versus miles) can "confuse" a Machine Learning Model and lead to some unexpected and unreliable predictions.

Using Standard Scaler to scale our data, we can bring our data in to a more readable place where our model can be better trained on the data and less sensitive to differences in numeric scale or measure.

To better understand StandardScaler, and to help you use it, it's a good idea to check out the documentation <a href='https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html'>here</a><br>
Note: Some code in this section borrowed from Tim Book and Matt Brems

In [40]:
sc = StandardScaler()

In [28]:
# Scale our data.
# Relabeling scaled data as "Z" is common.
Z_train = sc.fit_transform(X_train)
# DO NOT use 'fit' on your test data
Z_test = sc.transform(X_test)

In [ ]:
print(f'Z_train shape is: {Z_train.shape}')
print(f'y_train shape is: {y_train.shape}')
print(f'Z_test shape is: {Z_test.shape}')
print(f'y_test shape is: {y_test.shape}')

Now let's try running the scaled data through a new Logistic Regression model

In [ ]:
model_1 = LogisticRegression()
model_1.fit(Z_train, y_train)
print('Training Accuracy Score : ', model_1.score(Z_train, y_train))
print('Testing Accuracy Score  : ', model_1.score(Z_test, y_test))


## Holy Smokes!
We just improved our test accuracy by 35%, and our training accuracy by ~50%! Why is that?

In [ ]:
# type your answer here:

## What does the confusion matrix tell us?

In [ ]:
# Scale the X matrix into a Z matrix
Z = sc.transform(X)

# Show the predictions that the model makes on Z
y_pred = model_1.predict(Z)

# Evaluate the confusion matrix
confusion_matrix(y_true=y, y_pred=y_pred)

## Plot the confusion matrix into a visualization

In [ ]:

plot_confusion_matrix(model_1, Z, y, cmap='cividis')  
plt.show()  


What does the Confusion Matrix tell us?

[Hide this] We have 224 true positives and 89 false positives. We have 411 true negatives and 196 false negatives. That means that we erroneously classified 196 exoplanets as 'not-exoplanets'. Think of all the vacations that will never happen to those undiscovered planets if we didn't realize they were there. It also means that we classified 89 objects as 'exoplanets' when they were actually 'not-exoplanets'. That's a lot of rocket fuel to take us to nowhwere...

What are the following values:

Sensitivity (explain and compute)

In [ ]:
# type your answer here:

Recall (explain and compute)

In [ ]:
# type your answer here:

Specificity (explain and compute)

In [ ]:
# type your answer here:

Misclassification Rate (explain and compute)

In [ ]:
# type your answer here:

Precision (explain and compute)

In [ ]:
# type your answer here:

Is our model well-fit? Does it appear to show bias or over-fitting?

In [ ]:
# type your answer here:

What kinds of things might we do to improve our model?

In [ ]:
# type your answer here:

Should we call CalTech and NASA and tell them that we figured out which objects on the TESS Objects of Interest data set are Exoplanets?

In [ ]:
# type your answer here:

## Make Predictions
This is it, the moment you've been waiting for...time to make predictions on the unclassfied data

We're going to 

In [ ]:
tess_predictions = tess[tess['target'] == 2]
tess_predictions.head()

In [34]:
# Scale the prediction set's X matrix into a Z matrix
X_predictions = tess_predictions.drop(columns=['tfopwg_disp', 'target']).copy()
Z_predictions = sc.transform(X_predictions)



In [ ]:
tess_predictions['predictions'] = model_1.predict(Z_predictions)

In [ ]:
model_1.predict(Z_predictions)

In [ ]:
tess_predictions['predictions'].value_counts()

In [ ]:
tess_predictions['predictions'].value_counts(normalize=True)

<img src='https://photojournal.jpl.nasa.gov/jpeg/PIA22097.jpg'>
photo credit: https://www.jpl.nasa.gov/images/illustration-of-trappist-1-planets-as-of-feb-2018

## That's a lot of planets!
Your model has successfully made predictions. The predictions are a lot like the weather predictions in that you don't really have enough data to say that your model is 100% accurate, but your model is not like weather predictions in that weather predictions involve a LOT of data, multiple models strung together into ensemble models, and millions of data points.

Run the following cell to see ten examples of objects on the TESS Objects of Interest dataset that your model has positively identified. You can find these points in the sky by using the Right Ascension and Declination provided in the data table, and you can learn about the object if you'd like by noting the other observations recorded in the TOI.

In [ ]:
tess_predictions[tess_predictions['predictions']==1].sample(10)

## Conclusion
Your model just predicted the existence of exoplanets based on a dataset that is being put together by scientists at CalTech and NASA. Your predictions are only as good as your model, and if you didn't tweak your model beyond the base version of this notebook, then your accuracy score is only about 62%. You know that you'll have a high percentage of false predictions.

Once you've tweaked your model as much as you can, you should try running your data set through some other classification models such as the Random Forest Classifier, K Nearest Neighbors, or Support Vector Machines!